In [0]:
from pyspark.sql.functions import col, from_json, lit
from pyspark.sql.types import IntegerType, StringType, StructType

In [0]:
# conectando o spark streaming no tópico kafka
df = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', 'localhost:9092') \
    .option('subscribe', 'primeiro_topico') \
    .option('startingOffsets', 'earliest') \
    .load()

In [0]:
display(df)

key,value,topic,partition,offset,timestamp,timestampType
null,eyJpZCI6IDQsICJub21lIjogIkFsdW5vIDQiLCAiaWRhZGUiOiA0MH0=,primeiro_topico,0,0,2024-10-16T23:46:32.579Z,0
null,eyJpZCI6IDUsICJub21lIjogIkFsdW5vIDUiLCAiaWRhZGUiOiA1MH0=,primeiro_topico,0,1,2024-10-16T23:51:19.37Z,0
null,eyJpZCI6IDUwLCAibm9tZSI6ICJBbHVubyA1IiwgImlkYWRlIjogNTB9,primeiro_topico,0,2,2024-10-16T23:55:08.433Z,0
null,eyJpZCI6IDUwMCwgIm5vbWUiOiAiQWx1bm8gNTAwIiwgImlkYWRlIjogNTAwfQ==,primeiro_topico,0,3,2024-10-17T00:03:15.411Z,0
null,eyJpZCI6IDUwMDAsICJub21lIjogIkFsdW5vIDUwMCIsICJpZGFkZSI6IDUwMDB9,primeiro_topico,0,4,2024-10-17T00:09:55.133Z,0
null,eyJpZCI6IDUwMDAsICJub21lIjogIkFsdW5vIDUwMCIsICJpZGFkZSI6IDUwMDB9,primeiro_topico,0,5,2024-10-17T00:11:21.125Z,0
null,eyJpZCI6IDUwMDAsICJub21lIjogIkFsdW5vIDUwMCIsICJpZGFkZSI6IDUwMDB9,primeiro_topico,0,6,2024-10-17T00:18:29.396Z,0


In [0]:
#definir o esquema/estrutura dos campos do tópico primeiro_topico

schema = StructType() \
    .add('id', IntegerType()) \
    .add('nome', StringType()) \
    .add('idade', IntegerType())

In [0]:
# tradução da mensagem com o schema definido

df_resultado = df.selectExpr("CAST(value as STRING)") \
    .select(from_json(col('value'), schema).alias('data')) \
    .select('data.*')    

In [0]:
display(df_resultado)

id,nome,idade
4,Aluno 4,40
5,Aluno 5,50
50,Aluno 5,50
500,Aluno 500,500
5000,Aluno 500,5000
5000,Aluno 500,5000
5000,Aluno 500,5000


In [0]:
# criar coluna fixa
df_res = df_resultado.withColumn('nova_coluna', lit('Recebido pelo Spark Streaming'))

In [0]:
display(df_res)

id,nome,idade,nova_coluna
4,Aluno 4,40,Recebido pelo Spark Streaming
5,Aluno 5,50,Recebido pelo Spark Streaming
50,Aluno 5,50,Recebido pelo Spark Streaming
500,Aluno 500,500,Recebido pelo Spark Streaming
5000,Aluno 500,5000,Recebido pelo Spark Streaming
5000,Aluno 500,5000,Recebido pelo Spark Streaming
5000,Aluno 500,5000,Recebido pelo Spark Streaming


In [0]:
# ENVIAR INFORMAÇÃO PARA UM NOVO TÓPICO
# prepara a msg paar o envio em um novo tópico
df_res = df_res.selectExpr('CAST(id AS STRING) AS key', 'to_json(struct(*)) AS value')

In [0]:
df_res.writeStream \
    .format('kafka') \
    .outputMode('append') \
    .option('kafka.bootstrap.servers', 'localhost:9092') \
    .option('checkpointLocation', '/dbfs/tmp/_checkpoint') \
    .option('topic', 'topico_resultado_tratado') \
    .option("failOnDataLoss", "false") \
    .start() \
    .awaitTermination()

java.lang.Exception: Could not stop computations
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$4(DriverClient.scala:764)
	at scala.concurrent.Future.$anonfun$flatMap$1(Future.scala:307)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:54)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:77)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$2(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:273)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:269)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:46)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:43)
	at com.databricks.threading.ContextBoundRunnable.withAttributionContext(ContextBoundRunnable.scala:6)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$1(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.context.integrity.IntegrityCheckContext$ThreadLocalStorage$.withValue(IntegrityCheckContext.scala:73)
	at com.databricks.threading.ContextBoundRunnable.run(ContextBoundRunnable.scala:14)
	at com.databricks.threading.DatabricksExecutionContext$InstrumentedRunnable.run(DatabricksExecutionContext.scala:36)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:477)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)

In [0]:
df_res.writeStream \
    .format('parquet') \
    .option('path', '/dbfs/tmp/files_parquet/') \
    .option('checkpointLocation', 'dbfs/tmp/checkpoint') \
    .start() \
    .awaitTermination()